In [2]:
# Python
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient, Input, load_component, command
from azure.ai.ml.dsl import pipeline
from azure.ai.ml.entities import JobService

In [3]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

In [4]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

print(ml_client)

Found the config file in: /config.json


MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x7fcfca4f4dc0>,
         subscription_id=6025ba02-1dfd-407f-b358-88f811c7c7aa,
         resource_group_name=sc1-ml1,
         workspace_name=sc1ml1)


In [5]:
parent_dir = "."
download_model = load_component(source=parent_dir + "/model_download.yaml")
package_model = load_component(source=parent_dir + "/package_model.yaml")
update_model_env = load_component(source=parent_dir + "/update_model_env.yaml")

In [6]:
# Construct pipeline
@pipeline()
def import_model_pipeline(
registry: str = "azureml",
model_name: str = 'model',
model_version: str = 'latest'


):
    """E2E dummy train-score-eval pipeline with components defined via yaml."""
    # Call component obj as function: apply given inputs & parameters to create a node in pipeline
    download_model_output = download_model(
        registry=registry,
        model_name=model_name
    )
    """E2E dummy train-score-eval pipeline with components defined via yaml."""
    # Call component obj as function: apply given inputs & parameters to create a node in pipeline
    package_model_output = package_model(
        model_name=model_name,
        model_version=model_version,
        model_download=download_model_output.outputs.model_output
    )

    update_model_env_output = update_model_env(
        model_package=package_model_output.outputs.model_output
    )
    # Return: pipeline outputs
    return {
        "download_model": download_model_output.outputs.model_output,
        "package_model": package_model_output.outputs.model_output,
        "update_model_env": update_model_env_output.outputs.model_output
    }


pipeline_job = import_model_pipeline(
    registry='azureml',
    model_name='Deci-DeciLM-7B',
    model_version='latest'
)

# set pipeline level compute
pipeline_job.settings.default_compute = "model-download-cluster1"

In [7]:
# submit the command
returned_job = ml_client.create_or_update(pipeline_job)

Class AutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class AutoDeleteConditionSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseAutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class IntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class ProtectionLevelSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseIntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Uploading model_pipeline_src (0.03 MBs